In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore') 
from pylab import rcParams
rcParams["figure.figsize"]=30,16

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_excel("/content/drive/MyDrive/LCF_Internal/Renewable_Model/EDA Approach-1/XGBoost Implementation/v12/v_12.xlsx", sheet_name = 'ln_orig')
data.head()

,OpportunityID_ren,accountnumber,OpportunityID,lnlexidsele,inputcheckbusname,inputcheckbusaltname,inputcheckbusaddr,inputcheckbuscity,inputcheckbusstate,inputcheckbuszip,...,model1rc5,model1rc6,model2name,model2score,model2rc1,model2rc2,model2rc3,model2rc4,model2rc5,model2rc6
0,0062M00000gYsKAQA0,LCFLN202208126655,0062M00000gYsKAQA0,7.278946e+10,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,NaN,SLBO1702_0_2,691.0,B026,B052,B036,B037,NaN,NaN
1,0062M00000gYSqdQAG,LCFLN202208126660,0062M00000gYSqdQAG,0.000000e+00,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,NaN,SLBO1702_0_2,222.0,B068,NaN,NaN,NaN,NaN,NaN
2,0062M00000itNw6QAE,LCFLN202208127456,0062M00000itNw6QAE,7.087273e+08,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,NaN,SLBO1702_0_2,704.0,B069,B063,B052,B066,NaN,NaN
3,0062M00000ito3xQAA,LCFLN202208127457,0062M00000ito3xQAA,4.411074e+07,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,NaN,SLBO1702_0_2,692.0,B026,B069,B066,B076,NaN,NaN
4,0062M00000n0BgNQAU,LCFLN202208130968,0062M00000n0BgNQAU,1.345410e+11,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,NaN,SLBO1702_0_2,708.0,B069,B036,B052,B070,NaN,NaN


In [4]:
data.shape

(3689, 392)

In [5]:
'WriteOff_YN' in data.columns

False

In [6]:
# getting write_offs from another sheet i.e. renewal_orig
wo = pd.read_excel("/content/drive/MyDrive/LCF_Internal/Renewable_Model/EDA Approach-1/XGBoost Implementation/v12/v_12.xlsx",sheet_name = 'ren_orig')

In [7]:
'WriteOff_YN' in wo.columns

True

In [8]:
wo.shape

(3968, 55)

In [9]:
wo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3968 entries, 0 to 3967
Data columns (total 55 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   OpportunityID_ren                          3968 non-null   object        
 1   CreatedDate                                3968 non-null   datetime64[ns]
 2   FundedDate                                 3968 non-null   datetime64[ns]
 3   OpportunityName                            3968 non-null   object        
 4   Priordeal_OppID                            3968 non-null   object        
 5   Renewal_PreviousCollectionStatus           3968 non-null   object        
 6   Renewal_PreviousPace                       3953 non-null   float64       
 7   Renewal_PreviousPaidPercentageOfRTR        3953 non-null   float64       
 8   Renewal_PreviousCreditScore                3915 non-null   float64       
 9   Renewal_CountPrevio

In [10]:
# Considering Opportunity_ID and WriteOff_YN from Renewal_Exisiting_Attributes
wo = wo[['OpportunityID_ren','WriteOff_YN']]

In [11]:
wo

,OpportunityID_ren,WriteOff_YN
0,0062M00000o8ivqQAA,0.0
1,0062M00000hmHKAQA2,1.0
2,0062M00000kxeWLQAY,0.0
3,0062M00000mTGVnQAO,1.0
4,0062M00000gb1LRQAY,0.0
...,...,...
3963,0062M00000lWtjxQAC,0.0
3964,0062M00000bnAyFQAU,1.0
3965,0062M00000dioFlQAI,0.0
3966,0062M00000bngjMQAQ,1.0


In [12]:
d = pd.merge(data, wo, on="OpportunityID_ren")

In [13]:
d

,OpportunityID_ren,accountnumber,OpportunityID,lnlexidsele,inputcheckbusname,inputcheckbusaltname,inputcheckbusaddr,inputcheckbuscity,inputcheckbusstate,inputcheckbuszip,...,model1rc6,model2name,model2score,model2rc1,model2rc2,model2rc3,model2rc4,model2rc5,model2rc6,WriteOff_YN
0,0062M00000gYsKAQA0,LCFLN202208126655,0062M00000gYsKAQA0,7.278946e+10,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,SLBO1702_0_2,691.0,B026,B052,B036,B037,NaN,NaN,0.0
1,0062M00000gYSqdQAG,LCFLN202208126660,0062M00000gYSqdQAG,0.000000e+00,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,SLBO1702_0_2,222.0,B068,NaN,NaN,NaN,NaN,NaN,0.0
2,0062M00000itNw6QAE,LCFLN202208127456,0062M00000itNw6QAE,7.087273e+08,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,SLBO1702_0_2,704.0,B069,B063,B052,B066,NaN,NaN,0.0
3,0062M00000ito3xQAA,LCFLN202208127457,0062M00000ito3xQAA,4.411074e+07,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,SLBO1702_0_2,692.0,B026,B069,B066,B076,NaN,NaN,0.0
4,0062M00000n0BgNQAU,LCFLN202208130968,0062M00000n0BgNQAU,1.345410e+11,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,SLBO1702_0_2,708.0,B069,B036,B052,B070,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3684,0062M00000gYcpIQAS,LCFLN202208126604,0062M00000gYcpIQAS,1.337832e+11,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,SLBO1702_0_2,652.0,B026,B069,B034,B052,NaN,NaN,0.0
3685,0062M00000gYdrBQAS,LCFLN202208126606,0062M00000gYdrBQAS,1.335285e+11,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,SLBO1702_0_2,619.0,B026,B074,B063,B070,NaN,NaN,0.0
3686,0062M00000gYfg0QAC,LCFLN202208126608,0062M00000gYfg0QAC,5.107907e+07,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,SLBO1702_0_2,684.0,B026,B078,B063,B037,NaN,NaN,1.0
3687,0062M00000gYic7QAC,LCFLN202208126614,0062M00000gYic7QAC,2.199165e+07,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,SLBO1702_0_2,222.0,B068,NaN,NaN,NaN,NaN,NaN,1.0


In [14]:
d.columns

Index(['OpportunityID_ren', 'accountnumber', 'OpportunityID', 'lnlexidsele',
       'inputcheckbusname', 'inputcheckbusaltname', 'inputcheckbusaddr',
       'inputcheckbuscity', 'inputcheckbusstate', 'inputcheckbuszip',
       ...
       'model1rc6', 'model2name', 'model2score', 'model2rc1', 'model2rc2',
       'model2rc3', 'model2rc4', 'model2rc5', 'model2rc6', 'WriteOff_YN'],
      dtype='object', length=393)

In [15]:
data = d.copy()

In [16]:
data.shape

(3689, 393)

In [17]:
for i in data.columns:
  print(i)

OpportunityID_ren
accountnumber
OpportunityID
lnlexidsele
inputcheckbusname
inputcheckbusaltname
inputcheckbusaddr
inputcheckbuscity
inputcheckbusstate
inputcheckbuszip
inputcheckbusTIN
inputcheckbusphone
inputcheckbusfax
inputcheckbusnaics
inputcheckbussic
inputcheckbusage
inputcheckauthrepfirstname
inputcheckauthrepmiddlename
inputcheckauthreplastname
inputcheckauthrepaddr
inputcheckauthrepcity
inputcheckauthrepstate
inputcheckauthrepzip
inputcheckauthrepssn
inputcheckauthrepphone
inputcheckauthrepage
inputcheckauthrepdob
inputcheckauthrepdl
inputcheckauthrepdlstate
inputcheckauthrep2firstname
inputcheckauthrep2middlename
inputcheckauthrep2lastname
inputcheckauthrep2addr
inputcheckauthrep2city
inputcheckauthrep2state
inputcheckauthrep2zip
inputcheckauthrep2ssn
inputcheckauthrep2phone
inputcheckauthrep2age
inputcheckauthrep2dob
inputcheckauthrep2dl
inputcheckauthrep2dlstate
inputcheckauthrep3firstname
inputcheckauthrep3middlename
inputcheckauthrep3lastname
inputcheckauthrep3addr
input

In [18]:
bankruptcy = ['bankruptcycount', 'bankruptcycount12m',
              'bankruptcycount24m', 'bankruptcycount84m', 
              'bankruptcychapter', 'bankruptcytimenewest',
              'OpportunityID_ren', 'WriteOff_YN']
len(bankruptcy)

8

In [19]:
df = data[bankruptcy]

In [20]:
df.columns

Index(['bankruptcycount', 'bankruptcycount12m', 'bankruptcycount24m',
       'bankruptcycount84m', 'bankruptcychapter', 'bankruptcytimenewest',
       'OpportunityID_ren', 'WriteOff_YN'],
      dtype='object')

In [21]:
df.shape

(3689, 8)

In [22]:
df.describe()

,bankruptcycount,bankruptcycount12m,bankruptcycount24m,bankruptcycount84m,bankruptcychapter,bankruptcytimenewest,WriteOff_YN
count,3689.000000,3689.000000,3689.000000,3689.000000,3689.000000,3689.000000,3689.000000
mean,-0.134725,-0.144755,-0.141773,-0.135538,-0.042559,0.306587,0.265926
std,0.371884,0.354205,0.359581,0.370492,1.078572,5.184300,0.441885
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,11.000000,108.000000,1.000000


In [23]:
round(df.corr()['WriteOff_YN'].sort_values(),4)

bankruptcycount84m     -0.0216
bankruptcychapter      -0.0212
bankruptcycount        -0.0212
bankruptcycount24m     -0.0203
bankruptcycount12m     -0.0173
bankruptcytimenewest   -0.0122
WriteOff_YN             1.0000
Name: WriteOff_YN, dtype: float64

In [24]:
for i in df.columns:
  print('\033[1m'+'attribute_name :', i+'\033[0m')
  print(df[i].value_counts())

attribute_name : bankruptcycount
 0.0    3112
-1.0     537
 1.0      40
Name: bankruptcycount, dtype: int64
attribute_name : bankruptcycount12m
 0.0    3149
-1.0     537
 1.0       3
Name: bankruptcycount12m, dtype: int64
attribute_name : bankruptcycount24m
 0.0    3138
-1.0     537
 1.0      14
Name: bankruptcycount24m, dtype: int64
attribute_name : bankruptcycount84m
 0.0    3115
-1.0     537
 1.0      37
Name: bankruptcycount84m, dtype: int64
attribute_name : bankruptcychapter
 0.0     3112
-1.0      537
 11.0      25
 7.0       15
Name: bankruptcychapter, dtype: int64
attribute_name : bankruptcytimenewest
 0.0      3112
-1.0       537
 28.0        2
 41.0        2
 75.0        2
 15.0        2
 16.0        2
 31.0        1
 108.0       1
 6.0         1
 50.0        1
 21.0        1
 27.0        1
 13.0        1
 4.0         1
 20.0        1
 54.0        1
 61.0        1
 55.0        1
 58.0        1
 84.0        1
 104.0       1
 47.0        1
 53.0        1
 39.0        1
 45.0   

In [25]:
df.isna().sum()

bankruptcycount         0
bankruptcycount12m      0
bankruptcycount24m      0
bankruptcycount84m      0
bankruptcychapter       0
bankruptcytimenewest    0
OpportunityID_ren       0
WriteOff_YN             0
dtype: int64

In [26]:
df.shape

(3689, 8)

In [27]:
df.drop(columns =["bankruptcychapter"],axis = 1, inplace = True)

In [28]:
df.shape

(3689, 7)

In [29]:
df['bankruptcytimenewest_value'] = df["bankruptcytimenewest"].apply(lambda x:0 if x < 0 else x)
df['bankruptcytimenewest_flag'] = df["bankruptcytimenewest"].apply(lambda x:1 if x < 0 else 0)

In [31]:
df[df["OpportunityID_ren"] == '0062M00000itRUCQA2']["bankruptcytimenewest"]

142   -1.0
Name: bankruptcytimenewest, dtype: float64

In [33]:
df[df["OpportunityID_ren"] == '0062M00000noBRbQAM'][['bankruptcytimenewest', 'OpportunityID_ren']]

,bankruptcytimenewest,OpportunityID_ren
160,-1.0,0062M00000noBRbQAM


In [34]:
df.drop(columns = ["bankruptcytimenewest"], axis = 1, inplace=True)

In [35]:
df.columns

Index(['bankruptcycount', 'bankruptcycount12m', 'bankruptcycount24m',
       'bankruptcycount84m', 'OpportunityID_ren', 'WriteOff_YN',
       'bankruptcytimenewest_value', 'bankruptcytimenewest_flag'],
      dtype='object')

In [36]:
def bankruptcy_FE_categorical(df):
  df[['bankruptcycount_f1', 'bankruptcycount_f2', 'bankruptcycount_f3',
      'bankruptcycount12m_f1',"bankruptcycount12m_f2", "bankruptcycount12m_f3",
      'bankruptcycount24m_f1', 'bankruptcycount24m_f2', 'bankruptcycount24m_f3',
      'bankruptcycount84m_f1', 'bankruptcycount84m_f2',  'bankruptcycount84m_f3']] = 0
  
  for i in ['bankruptcycount', 'bankruptcycount12m', 'bankruptcycount24m',
            'bankruptcycount84m']:
        df[i+"_f1"] = df[i].apply(lambda x: 1 if x== -1 else 0)
        df[i+"_f2"] = df[i].apply(lambda x: 1 if x == 0 else 0)
        df[i+"_f3"] = df[i].apply(lambda x: 1 if x == 1 else 0)
        df.drop(i, axis = 'columns', inplace = True)

In [37]:
# function call 
bankruptcy_FE_categorical(df)

In [38]:
df.shape

(3689, 16)

In [39]:
# basic function check
df.columns

Index(['OpportunityID_ren', 'WriteOff_YN', 'bankruptcytimenewest_value',
       'bankruptcytimenewest_flag', 'bankruptcycount_f1', 'bankruptcycount_f2',
       'bankruptcycount_f3', 'bankruptcycount12m_f1', 'bankruptcycount12m_f2',
       'bankruptcycount12m_f3', 'bankruptcycount24m_f1',
       'bankruptcycount24m_f2', 'bankruptcycount24m_f3',
       'bankruptcycount84m_f1', 'bankruptcycount84m_f2',
       'bankruptcycount84m_f3'],
      dtype='object')

In [40]:
round(df.corr()['WriteOff_YN'].sort_values(),4)

bankruptcycount12m_f2        -0.0215
bankruptcycount24m_f2        -0.0180
bankruptcycount_f2           -0.0161
bankruptcycount84m_f2        -0.0158
bankruptcycount84m_f3        -0.0113
bankruptcytimenewest_value   -0.0109
bankruptcycount_f3           -0.0097
bankruptcycount24m_f3        -0.0072
bankruptcytimenewest_flag     0.0195
bankruptcycount_f1            0.0195
bankruptcycount12m_f1         0.0195
bankruptcycount24m_f1         0.0195
bankruptcycount84m_f1         0.0195
bankruptcycount12m_f3         0.0259
WriteOff_YN                   1.0000
Name: WriteOff_YN, dtype: float64

In [41]:
df.to_excel("/content/drive/MyDrive/LCF_Internal/Renewable_Model/EDA Approach-1/XGBoost Implementation/v12/FE_V12_CSVs/bankruptcy.xlsx", index = False)

# Firmographic Attributes

In [42]:
firmographic_attributes = ['firmnonprofit', 'firmirsretirementplan', 'firmageobserved', 'firmemployeecount',
                          'firmemployeerangecount', 'firmownershiptype', 'firmreportedsales', 'firmreportedsalesrange',
                          'firmreportedearnings', 'firmparentcompanyind', 'OpportunityID_ren', 'WriteOff_YN']
len(firmographic_attributes)

12

In [43]:
df = data[firmographic_attributes]
df.head(3)

,firmnonprofit,firmirsretirementplan,firmageobserved,firmemployeecount,firmemployeerangecount,firmownershiptype,firmreportedsales,firmreportedsalesrange,firmreportedearnings,firmparentcompanyind,OpportunityID_ren,WriteOff_YN
0,0.0,0.0,5.0,0.0,0.0,0.0,408000.0,50.0,-1.0,1.0,0062M00000gYsKAQA0,0.0
1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0062M00000gYSqdQAG,0.0
2,0.0,0.0,12.0,2.0,20.0,0.0,557000.0,60.0,-1.0,1.0,0062M00000itNw6QAE,0.0


In [44]:
df.columns

Index(['firmnonprofit', 'firmirsretirementplan', 'firmageobserved',
       'firmemployeecount', 'firmemployeerangecount', 'firmownershiptype',
       'firmreportedsales', 'firmreportedsalesrange', 'firmreportedearnings',
       'firmparentcompanyind', 'OpportunityID_ren', 'WriteOff_YN'],
      dtype='object')

In [45]:
df.shape

(3689, 12)

In [46]:
df.describe()

,firmnonprofit,firmirsretirementplan,firmageobserved,firmemployeecount,firmemployeerangecount,firmownershiptype,firmreportedsales,firmreportedsalesrange,firmreportedearnings,firmparentcompanyind,WriteOff_YN
count,3689.000000,3689.000000,3689.000000,3689.000000,3689.000000,3689.000000,3.689000e+03,3689.000000,3689.000000,3689.000000,3689.000000
mean,-0.136622,-0.135267,7.039035,2.604229,11.125779,-0.145568,3.242394e+05,18.694497,278.791542,0.708864,0.265926
std,0.368625,0.370957,11.056260,7.276611,15.130861,0.352720,1.062483e+06,26.604523,9958.110452,0.705441,0.441885
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000e+00,-1.000000,-1.000000,-1.000000,0.000000
25%,0.000000,0.000000,1.000000,-1.000000,-1.000000,0.000000,-1.000000e+00,-1.000000,-1.000000,1.000000,0.000000
50%,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,-1.000000e+00,-1.000000,-1.000000,1.000000,0.000000
75%,0.000000,0.000000,9.000000,3.000000,20.000000,0.000000,2.050000e+05,40.000000,-1.000000,1.000000,1.000000
max,1.000000,1.000000,110.000000,100.000000,70.000000,0.000000,2.299900e+07,100.000000,499510.000000,1.000000,1.000000


In [47]:
round(df.corr()['WriteOff_YN'].sort_values(),4)

firmageobserved          -0.0976
firmemployeerangecount   -0.0860
firmreportedsalesrange   -0.0705
firmemployeecount        -0.0415
firmirsretirementplan    -0.0253
firmnonprofit            -0.0216
firmownershiptype        -0.0195
firmparentcompanyind     -0.0195
firmreportedsales        -0.0155
firmreportedearnings      0.0095
WriteOff_YN               1.0000
Name: WriteOff_YN, dtype: float64

In [48]:
for i in df.columns:
  print('\033[1m'+'attribute_name :', i+'\033[0m')
  print(df[i].value_counts())

attribute_name : firmnonprofit
 0.0    3119
-1.0     537
 1.0      33
Name: firmnonprofit, dtype: int64
attribute_name : firmirsretirementplan
 0.0    3114
-1.0     537
 1.0      38
Name: firmirsretirementplan, dtype: int64
attribute_name : firmageobserved
 2.0      566
-1.0      549
 1.0      442
 3.0      349
 4.0      246
         ... 
 74.0       1
 54.0       1
 103.0      1
 75.0       1
 85.0       1
Name: firmageobserved, Length: 64, dtype: int64
attribute_name : firmemployeecount
-1.0      1777
 2.0       417
 1.0       368
 0.0       170
 3.0       166
 4.0       118
 5.0        90
 6.0        73
 9.0        59
 7.0        58
 10.0       51
 13.0       41
 8.0        38
 11.0       30
 12.0       26
 18.0       25
 14.0       18
 15.0       17
 23.0       14
 32.0       13
 25.0       13
 21.0       11
 17.0       10
 19.0        8
 16.0        8
 22.0        6
 30.0        6
 26.0        5
 50.0        5
 20.0        4
 33.0        4
 41.0        4
 40.0        4
 31.0      

In [49]:
df.isna().sum()

firmnonprofit             0
firmirsretirementplan     0
firmageobserved           0
firmemployeecount         0
firmemployeerangecount    0
firmownershiptype         0
firmreportedsales         0
firmreportedsalesrange    0
firmreportedearnings      0
firmparentcompanyind      0
OpportunityID_ren         0
WriteOff_YN               0
dtype: int64

In [50]:
df.shape

(3689, 12)

In [51]:
# There are 3255 missing values in this feature
df.drop(['firmreportedearnings', 'firmreportedsales', 'firmemployeecount'], axis = 1, inplace = True)

In [52]:
df.shape

(3689, 9)

In [53]:
def firmographic_attributes_FE(df):
  for i in ['firmageobserved']:
      
      df[i+'_value'] = df[i].apply(lambda x: 0 if x < 0 else x)
      df[i+'_flag'] = df[i].apply(lambda x: 1 if x < 0 else 0)
      df.drop(i,axis = 'columns',inplace = True)

In [54]:
def firmographic_FE_categorical(df):
  df[['firmnonprofit_f1', 'firmnonprofit_f2', 'firmnonprofit_f3',
      'firmirsretirementplan_f1',"firmirsretirementplan_f2", "firmirsretirementplan_f3",
      'firmownershiptype_f1', 'firmownershiptype_f2', 'firmownershiptype_f3',
      'firmparentcompanyind_f1', 'firmparentcompanyind_f2',  'firmparentcompanyind_f3']] = 0
  
  for i in ['firmnonprofit', 'firmirsretirementplan', 'firmownershiptype',
            'firmparentcompanyind']:
        df[i+"_f1"] = df[i].apply(lambda x: 1 if x== -1 else 0)
        df[i+"_f2"] = df[i].apply(lambda x: 1 if x == 0 else 0)
        df[i+"_f3"] = df[i].apply(lambda x: 1 if x == 1 else 0)
        df.drop(i, axis = 'columns', inplace = True)

In [ ]:
# attribute_name : firmownershiptype
#  0.0    2874
# -1.0     508

# attribute_name : firmparentcompanyind
#  1.0    2874
# -1.0     508

In [55]:
df = pd.get_dummies(df, columns = ['firmemployeerangecount', 'firmreportedsalesrange'])
df.shape

(3689, 29)

In [56]:
firmographic_attributes_FE(df)
firmographic_FE_categorical(df)
df.head()

,OpportunityID_ren,WriteOff_YN,firmemployeerangecount_-1.0,firmemployeerangecount_0.0,firmemployeerangecount_10.0,firmemployeerangecount_20.0,firmemployeerangecount_30.0,firmemployeerangecount_40.0,firmemployeerangecount_50.0,firmemployeerangecount_60.0,...,firmnonprofit_f3,firmirsretirementplan_f1,firmirsretirementplan_f2,firmirsretirementplan_f3,firmownershiptype_f1,firmownershiptype_f2,firmownershiptype_f3,firmparentcompanyind_f1,firmparentcompanyind_f2,firmparentcompanyind_f3
0,0062M00000gYsKAQA0,0.0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
1,0062M00000gYSqdQAG,0.0,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
2,0062M00000itNw6QAE,0.0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
3,0062M00000ito3xQAA,0.0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
4,0062M00000n0BgNQAU,0.0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1


In [57]:
df.shape

(3689, 38)

In [58]:
df.columns

Index(['OpportunityID_ren', 'WriteOff_YN', 'firmemployeerangecount_-1.0',
       'firmemployeerangecount_0.0', 'firmemployeerangecount_10.0',
       'firmemployeerangecount_20.0', 'firmemployeerangecount_30.0',
       'firmemployeerangecount_40.0', 'firmemployeerangecount_50.0',
       'firmemployeerangecount_60.0', 'firmemployeerangecount_70.0',
       'firmreportedsalesrange_-1.0', 'firmreportedsalesrange_0.0',
       'firmreportedsalesrange_5.0', 'firmreportedsalesrange_10.0',
       'firmreportedsalesrange_20.0', 'firmreportedsalesrange_30.0',
       'firmreportedsalesrange_40.0', 'firmreportedsalesrange_50.0',
       'firmreportedsalesrange_60.0', 'firmreportedsalesrange_70.0',
       'firmreportedsalesrange_80.0', 'firmreportedsalesrange_90.0',
       'firmreportedsalesrange_100.0', 'firmageobserved_value',
       'firmageobserved_flag', 'firmnonprofit_f1', 'firmnonprofit_f2',
       'firmnonprofit_f3', 'firmirsretirementplan_f1',
       'firmirsretirementplan_f2', 'firmirsretire

In [59]:
round(df.corr()['WriteOff_YN'].sort_values(),4)

firmageobserved_value          -0.0980
firmemployeerangecount_30.0    -0.0559
firmreportedsalesrange_70.0    -0.0548
firmreportedsalesrange_40.0    -0.0458
firmreportedsalesrange_50.0    -0.0347
firmemployeerangecount_40.0    -0.0333
firmemployeerangecount_50.0    -0.0329
firmemployeerangecount_20.0    -0.0319
firmemployeerangecount_10.0    -0.0263
firmreportedsalesrange_10.0    -0.0254
firmirsretirementplan_f3       -0.0249
firmownershiptype_f2           -0.0195
firmparentcompanyind_f3        -0.0195
firmnonprofit_f2               -0.0160
firmirsretirementplan_f2       -0.0120
firmnonprofit_f3               -0.0116
firmreportedsalesrange_20.0    -0.0074
firmreportedsalesrange_80.0    -0.0067
firmreportedsalesrange_30.0    -0.0064
firmreportedsalesrange_100.0   -0.0017
firmreportedsalesrange_60.0     0.0008
firmreportedsalesrange_5.0      0.0075
firmreportedsalesrange_0.0      0.0102
firmemployeerangecount_0.0      0.0111
firmageobserved_flag            0.0155
firmnonprofit_f1         

In [60]:
df.drop(columns = ["firmownershiptype_f3","firmparentcompanyind_f2"], axis = 1,inplace=True)

In [62]:
df.to_excel("/content/drive/MyDrive/LCF_Internal/Renewable_Model/EDA Approach-1/XGBoost Implementation/v12/FE_V12_CSVs/firmography.xlsx", index = False)